# Baseline LDA model
This notebook gives an overview of how to train an LDA model from the Reddit data.

The input is a joined submissions and commends dataframe as produced by `notebooks/bagOfWords_preprocessing_databricks.ipynb`

In [1]:
import ihop.utils
spark = ihop.utils.get_spark_session("baseline lda")

input_data = spark.read.load(
    "../data/bagOfWords/2021-05_to_2021-06_joined_submissions_comments_5percentTopUsersExcludedFromComments_02102022.parquet").limit(200)


22/02/22 13:43:27 WARN Utils: Your hostname, Kurt resolves to a loopback address: 127.0.1.1; using 192.168.0.11 instead (on interface wlp4s0)
22/02/22 13:43:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/22 13:43:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/22 13:43:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark configuration:
[('spark.app.startTime', '1645555408216'), ('spark.driver.port', '35221'), ('spark.app.id', 'local-1645555410249'), ('spark.executor.id', 'driver'), ('spark.app.name', 'baseline lda'), ('spark.driver.memory', '8G'), ('spark.driver.host', '192.168.0.11'), ('spark.sql.warehouse.dir', 'file:/home/virginia/Documents/CenterForDataScience/ZuckermanProj/IHOP/notebooks/spark-warehouse'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.executor.extraLibraryPath', '/home/virginia/hadoop-3.3.1/lib/native'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.extraLibraryPath', '/home/virginia/hadoop-3.3.1/lib/native')]


In [2]:
input_data.show(5)

In [ ]:
input_data.count()

# Train a simple LDA model using Gensim


In [ ]:
from datetime import datetime
import ihop.clustering as ic
import ihop.text_processing as itp

# Read in the joined data, collecting all the comments for each submission
# Any desired filtering by time stamps can be done here
corpus = itp.SparkCorpus.init_from_joined_dataframe(input_data)

# Tokenize the document, then create an id to word index and vectorize each document
# This is where you would set minimum and maximum document frequency and minimum term frequency, passed to Spark CountVectorizer
pipeline = itp.SparkTextPreprocessingPipeline('document_text')
transformed = pipeline.fit_transform(corpus.document_dataframe)
vectorized_corpus = itp.SparkCorpus(transformed)


In [ ]:
index = pipeline.get_id_to_word()
print("INDEX DETAILS:")
print("Vocab size:", len(index))
for k in range(10):
    print(k, index[k])

In [ ]:
vectorized_corpus.document_dataframe.show()

In [ ]:
corpus_iterator = vectorized_corpus.get_vectorized_column_iterator()

In [ ]:
lda_model = ic.GensimLDAModel(corpus_iterator, "sample_lda", index, num_topics=10)

print("Starting training at", datetime.now())
lda_model.train()

In [ ]:
lda_model.get_top_words_as_dataframe()

In [ ]:
lda_model.get_top_words()

In [ ]:
print(lda_model.get_metrics())

In [ ]:
lda_model.get_term_topics('i')

In [ ]:
lda_model.get_parameters()

In [ ]:
id_corpus_iter = vectorized_corpus.get_vectorized_column_iterator(use_id_col=True)
doc_topics = lda_model.get_topic_assigments(id_corpus_iter)
print(len(doc_topics))
doc_topics

In [ ]:
subreddits = corpus.document_dataframe.select('id', 'subreddit').toPandas()
topics_df = lda_model.get_cluster_results_as_df(vocab_col_name="id", join_df = subreddits)
topics_df